In [1]:
!git clone https://github.com/anvinhluong/Vietnamese-text-readability.git

Cloning into 'Vietnamese-text-readability'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (9/9), 6.24 MiB | 17.09 MiB/s, done.


In [2]:
!unzip /content/Vietnamese-text-readability/data.zip -d /content/data


Archive:  /content/Vietnamese-text-readability/data.zip
   creating: /content/data/Vietnamese Text Readability Corpus/
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_001.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_002.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_003.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_004.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_005.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_006.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_007.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_008.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_009.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus/difficult_010.txt  
  inflating: /content/data/Vietnamese Text Readability Corpus

In [5]:
import glob
import os
from transformers import BertTokenizer

# Set up directories and file patterns
print("Current working directory:", os.getcwd())
os.chdir('/content/data/copus')

# Find all files
difficult_file = glob.glob('difficult_*.txt')
easy_file = glob.glob('easy_*.txt')
medium_file = glob.glob('medium_*.txt')
veryeasy_file = glob.glob('veryeasy_*.txt')

# Create lists to store file contents
all_contents = []
difficult_content = []
easy_content = []
medium_content = []
veryeasy_content = []

# Read and store file contents
for file_name in difficult_file:
    with open(file_name, 'r', encoding='utf-8-sig') as file:
        content = file.read()
        all_contents.append(content)
        difficult_content.append(content)

for file_name in easy_file:
    with open(file_name, 'r', encoding='utf-8-sig') as file:
        content = file.read()
        all_contents.append(content)
        easy_content.append(content)

for file_name in medium_file:
    with open(file_name, 'r', encoding='utf-8-sig') as file:
        content = file.read()
        all_contents.append(content)
        medium_content.append(content)

for file_name in veryeasy_file:
    with open(file_name, 'r', encoding='utf-8-sig') as file:
        content = file.read()
        all_contents.append(content)
        veryeasy_content.append(content)

print(len(difficult_content))
print(len(easy_content))
print(len(medium_content))
print(len(veryeasy_content))
print(len(all_contents))



Current working directory: /content/data/copus
321
453
242
809
1825


In [6]:
from transformers import AutoTokenizer, BertModel

# Tải tokenizer PhoBERT
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

# Mã hóa các đoạn văn bản thành các token
inputs_easy = tokenizer(easy_content, return_tensors='pt', padding=True, truncation=False)
inputs_diff = tokenizer(difficult_content, return_tensors='pt', padding=True, truncation=False)
inputs_medium = tokenizer(medium_content, return_tensors='pt', padding=True, truncation=False)
inputs_veryeasy = tokenizer(veryeasy_content, return_tensors='pt', padding=True, truncation=False)

# Lấy các token id và attention mask
token_diff = inputs_diff['input_ids']
token_easy = inputs_easy['input_ids']

token_medium = inputs_medium['input_ids']
token_veryeasy = inputs_veryeasy['input_ids']



# print(token_diff)
# print(token_easy)
# print(token_medium)
# print(token_veryeasy)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [8]:
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import torch
y_diff=[]
y_medium=[]
y_easy=[]
y_veryeasy=[]
for i in token_diff:
    y_diff.append([1, 0, 0,0])
y_diff = np.array(y_diff)
for i in token_medium:
    y_medium.append([0, 1, 0,0])
y_medium = np.array(y_medium)
for i in token_easy:
    y_easy.append([0, 0, 1,0])
y_easy = np.array(y_easy)
for i in token_veryeasy:
    y_veryeasy.append([0, 0, 0,1])
y_veryeasy = np.array(y_veryeasy)
train_diff, test_diff, y_train_diff, y_test_diff = train_test_split(token_diff, y_diff, test_size=0.2, random_state=42)
train_medium, test_medium, y_train_medium,y_test_medium = train_test_split(token_medium, y_medium, test_size=0.1, random_state=42)
train_easy, test_easy, y_train_easy,y_test_easy = train_test_split(token_easy, y_easy, test_size=0.2, random_state=42)
train_veryeasy, test_veryeasy, y_train_veryeasy,y_test_veryeasy = train_test_split(token_veryeasy, y_veryeasy, test_size=0.2, random_state=42)

train_data=[train_diff,train_medium,train_easy,train_veryeasy]

test_data=[test_diff,test_easy,test_medium,test_veryeasy]

y_train=[np.array(y_train_diff),np.array(y_train_medium),np.array(y_test_easy),np.array(y_train_veryeasy)]
y_train = np.vstack(y_train)
y_test=[y_test_diff,y_test_easy,y_test_medium,y_test_veryeasy]


#format data
#train
max_length = max(tensor.size(1) for tensor in train_data)

# Hàm để padding các tensor
def pad_tensor(tensor, max_length):
    pad_size = max_length - tensor.size(1)
    return torch.nn.functional.pad(tensor, (0, pad_size), mode='constant', value=0)

# Thực hiện padding cho tất cả các tensor
padded_tensors = [pad_tensor(tensor, max_length) for tensor in train_data]
train_dataset = torch.cat(padded_tensors, dim=0)


#test
max_length_test = max(tensor.size(1) for tensor in test_data)
padded_tensors_test = [pad_tensor(tensor, max_length_test) for tensor in test_data]
test_dataset = torch.cat(padded_tensors_test, dim=0)


In [9]:
import torch
import torch.nn as nn
from transformers import BertModel

# Khởi tạo PhoBERTBASE
bert_model = BertModel.from_pretrained('vinai/phobert-base')

def chunk_tokens(tokens, chunk_size):
    tokens = tokens.tolist()  # Convert the tensor to a list

    if len(tokens) <= chunk_size:
        tokens = tokens + [0] * (chunk_size - len(tokens))
        return [tokens]
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Nếu phần cuối cùng không đủ chunk_size, thực hiện padding
    if len(chunks[-1]) < chunk_size:
        chunks[-1] = chunks[-1] + [0] * (chunk_size - len(chunks[-1]))

    return chunks

def create_attention_mask(chunks, chunk_size=256):
    attention_masks = []
    for chunk in chunks:
        attention_mask = [1] * len(chunk) + [0] * (chunk_size - len(chunk))
        attention_masks.append(attention_mask[:chunk_size])
    return torch.tensor(attention_masks)

def get_bert_features(model, input_ids, attention_mask):
    with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    return outputs.last_hidden_state

class BERTTextClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(BERTTextClassifier, self).__init__()
        self.bert = bert_model
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, num_classes)  # Đảm bảo số lớp đầu ra là num_classes
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_ids):
        chunk_size = 256
        batch_size = input_ids.size(0)
        device = input_ids.device
        all_features = []

        for i in range(batch_size):
            chunks = chunk_tokens(input_ids[i], chunk_size)
            attention_masks = create_attention_mask(chunks, chunk_size)
            batch_features = []

            for chunk, attention_mask in zip(chunks, attention_masks):
                chunk_tensor = torch.tensor(chunk, dtype=torch.long).unsqueeze(0).to(device)
                attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0).to(device)
                features = get_bert_features(self.bert, chunk_tensor, attention_mask_tensor)
                batch_features.append(features.squeeze(0))

            avg_features = torch.mean(torch.stack(batch_features), dim=0)
            all_features.append(avg_features)

        avg_features_batch = torch.stack(all_features)
        x = torch.mean(avg_features_batch, dim=1)
        x = self.dense(x)
        x = self.relu(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x


hidden_size = 768  # Kích thước vector đặc trưng từ PhoBERT
num_classes = 4    # Số lượng lớp phân loại
model = BERTTextClassifier(bert_model, hidden_size, num_classes)


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.

In [10]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):

        if idx >= len(self.x_data):
            raise IndexError("Index out of bounds")
        return self.x_data[idx], self.y_data[idx]


In [11]:
from torch.utils.data import DataLoader

# Khởi tạo dataset và dataloader
x_train_tensor = torch.tensor(train_dataset, dtype=torch.long)  # Chọn dtype phù hợp
print(y_train)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
print(y_train_tensor)
train_dataset_custom = CustomDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset_custom, batch_size=2, shuffle=True)

<ipython-input-11-41c69d388723>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_tensor = torch.tensor(train_dataset, dtype=torch.long)  # Chọn dtype phù hợp


[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])


In [12]:
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader

# Khởi tạo device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

def train_epoch(model, train_loader, criterion, optimizer, device):

    model.train()
    total_loss = 0
    for input_ids, labels in train_loader:

        input_ids = input_ids.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataset)
    print(f'Average training loss: {avg_loss}')

for epoch in range(5):  # Số lượng epoch có thể thay đổi
    print(f'Epoch {epoch + 1}/{5}')
    y_train_tensor = [torch.tensor(arr, dtype=torch.float32) for arr in y_train]
    train_epoch(model, train_loader, criterion, optimizer, device)


Epoch 1/5


<ipython-input-9-c904e444f4da>:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long).unsqueeze(0).to(device)


IndexError: index 1252 is out of bounds for dimension 0 with size 1211

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

# Khởi tạo PhoBERTBASE
bert_model = BertModel.from_pretrained('vinai/phobert-base')

def chunk_tokens(tokens, chunk_size):
    tokens = tokens.tolist()  # Convert the tensor to a list

    if len(tokens) <= chunk_size:
        tokens = tokens + [0] * (chunk_size - len(tokens))
        return [tokens]
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Nếu phần cuối cùng không đủ chunk_size, thực hiện padding
    if len(chunks[-1]) < chunk_size:
        chunks[-1] = chunks[-1] + [0] * (chunk_size - len(chunks[-1]))

    return chunks

def create_attention_mask(chunks, chunk_size=256):
    attention_masks = []
    for chunk in chunks:
        attention_mask = [1] * len(chunk) + [0] * (chunk_size - len(chunk))
        attention_masks.append(attention_mask[:chunk_size])
    return torch.tensor(attention_masks)

def get_bert_features(model, input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    return outputs.last_hidden_state

class BERTTextClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, num_classes):
        super(BERTTextClassifier, self).__init__()
        self.bert = bert_model
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()  # Thay thế Softmax với Sigmoid

    def forward(self, input_ids):
        chunk_size = 256
        chunks = chunk_tokens(input_ids, chunk_size)
        attention_masks = create_attention_mask(chunks, chunk_size)

        all_features = []
        for chunk, attention_mask in zip(chunks, attention_masks):
            chunk_tensor = torch.tensor(chunk).unsqueeze(0).to(input_ids.device)
            attention_mask_tensor = torch.tensor(attention_mask).unsqueeze(0).to(input_ids.device)
            features = get_bert_features(self.bert, chunk_tensor, attention_mask_tensor)
            all_features.append(features.squeeze(0))

        # Tính trung bình các đặc trưng của tất cả các đoạn
        avg_features = torch.mean(torch.stack(all_features), dim=0)
        x = self.dense(avg_features.mean(dim=0).unsqueeze(0))
        x = self.fc(self.relu(x))
        x = self.sigmoid(x)  # Thay đổi để sử dụng sigmoid cho phân loại nhị phân
        return x

hidden_size = 768  # Kích thước vector đặc trưng từ PhoBERT
num_classes = 4    # Số lượng lớp phân loại
model = BERTTextClassifier(bert_model, hidden_size, num_classes)

# Ví dụ về nhãn one-hot encoding
labels = torch.tensor([1, 0, 0, 0], dtype=torch.float32)  # One-hot encoding cho một ví dụ

# Dự đoán của mô hình
input_ids = torch.randint(0, 1000, (256,))  # Ví dụ về đầu vào
outputs = model(input_ids)

# Loss function
criterion = nn.BCEWithLogitsLoss()
loss = criterion(outputs.squeeze(0), labels)
print("Loss:", loss.item())


In [22]:
import numpy as np
import torch

# Danh sách các mảng numpy
arrays = [np.array([[1, 0, 0, 0]]), np.array([[0, 1, 0, 0]]), np.array([[0, 0, 1, 0]]), np.array([[0, 0, 0, 1]])]

# Chuyển đổi danh sách thành một mảng numpy duy nhất
numpy_array = np.vstack(arrays)

# Chuyển đổi mảng numpy thành tensor PyTorch
tensor = torch.tensor(numpy_array, dtype=torch.float32)

print(tensor)


tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
